##### Installation

In [4]:
!pip install torch
!pip install tensorflow
!pip install tensorflow-datasets
!pip install transformers

# for tracking and visualising training in web browser
!pip install wandb
!wandb login 20983d37439737ea3e750cd826d32f9ed826c1dc

     |████████████████████████████████| 675kB 7.3MB/s 
     |████████████████████████████████| 1.1MB 18.6MB/s 
     |████████████████████████████████| 3.8MB 38.6MB/s 
     |████████████████████████████████| 890kB 28.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=16762286a397ab70d7ba405967b0b5615fd9b721ca42998856a179aa717aa74f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.4MB 7.1MB/s 
     |████████████████████████████████| 102kB 10.4MB/s 
     |████████████████████████████████| 102kB 10.9MB/s 
     |████████████████████████████████| 112kB 37.1MB/s 
     |████████████████████████████████| 460kB 37.6MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 71kB 8.6MB/s 
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=35cf169672070

In [5]:
%tensorflow_version 2.x
from datetime import datetime
from google.colab import files

In [6]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [ ]:
data_file_name = data_express_original.txt

In [7]:
%mkdir /content/data
%cp -av /gdrive/My\ Drive/data/$data_file_name /content/data/

'/gdrive/My Drive/data/data_express_original.txt' -> '/content/data/data_express_original.txt'


In [8]:
!ls

data  sample_data  wandb


In [9]:
!nvidia-smi

Sat Jun 27 02:11:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##### Training


In [10]:
import tensorflow as tf
import tensorflow_datasets
from transformers import *

# Load dataset, tokenizer, model from pretrained model/vocabulary
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-cased')
data = tensorflow_datasets.load('glue/mrpc')

# Prepare dataset for GLUE as a tf.data.Dataset instance
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task='mrpc')
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task='mrpc')
train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
valid_dataset = valid_dataset.batch(64)

# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train and evaluate using tf.keras.Model.fit()
history = model.fit(train_dataset, epochs=2, steps_per_epoch=115,
                    validation_data=valid_dataset, validation_steps=7)

# Load the TensorFlow model in PyTorch for inspection
model.save_pretrained('./data/')
pytorch_model = BertForSequenceClassification.from_pretrained('./data/', from_tf=True)

# Quickly test a few predictions - MRPC is a paraphrasing task, let's see if our model learned the task
sentence_0 = "This research was consistent with his findings."
sentence_1 = "His findings were compatible with this research."
sentence_2 = "His findings were not compatible with this research."
inputs_1 = tokenizer.encode_plus(sentence_0, sentence_1, add_special_tokens=True, return_tensors='pt')
inputs_2 = tokenizer.encode_plus(sentence_0, sentence_2, add_special_tokens=True, return_tensors='pt')

pred_1 = pytorch_model(inputs_1['input_ids'], token_type_ids=inputs_1['token_type_ids'])[0].argmax().item()
pred_2 = pytorch_model(inputs_2['input_ids'], token_type_ids=inputs_2['token_type_ids'])[0].argmax().item()

print("sentence_1 is", "a paraphrase" if pred_1 else "not a paraphrase", "of sentence_0")
print("sentence_2 is", "a paraphrase" if pred_2 else "not a paraphrase", "of sentence_0")

KeyboardInterrupt: ignored

In [18]:
!mkdir train

In [11]:
from transformers import TransfoXLTokenizer, TFTransfoXLModel
import torch
import tensorflow as tf
import tensorflow_datasets as tfds

import os

###### Convert .txt file to tf.data.Dataset

In [12]:
file_name = 'data/data_express_original.txt'

In [13]:
pretrained_weight = 'transfo-xl-wt103'
tokenizer = TransfoXLTokenizer.from_pretrained(pretrained_weight)
model = TFTransfoXLModel.from_pretrained(pretrained_weight)

In [19]:
from pathlib import Path

paths = [str(x) for x in Path(".").glob("data/data_express_transpose_original.txt")] #glob("data/*.txt")
print(paths)

tokenizer.train(files=paths)

[]


TypeError: ignored

In [21]:
from tokenizers import Tokenizer
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from tokenizers.models import BPE
from tokenizers.normalizers import Lowercase, NFKC, Sequence
from tokenizers.pre_tokenizers import ByteLevel

# First we create an empty Byte-Pair Encoding model (i.e. not trained model)
tokenizer = Tokenizer(BPE())

# Then we enable lower-casing and unicode-normalization
# The Sequence normalizer allows us to combine multiple Normalizer that will be
# executed in order.
tokenizer.normalizer = Sequence([
    NFKC(),
    Lowercase()
])

# Our tokenizer also needs a pre-tokenizer responsible for converting the input to a ByteLevel representation.
tokenizer.pre_tokenizer = ByteLevel()

# And finally, let's plug a decoder so we can recover from a tokenized input to the original one
tokenizer.decoder = ByteLevelDecoder()


from tokenizers.trainers import BpeTrainer

# We initialize our trainer, giving him the details about the vocabulary we want to generate
trainer = BpeTrainer(vocab_size=25000, show_progress=True, initial_alphabet=ByteLevel.alphabet())
tokenizer.train(trainer, ["./data/data_express_original.txt"])

print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 521


In [22]:
# You will see the generated files in the output.
tokenizer.model.save('./train/')

['./train/vocab.json', './train/merges.txt']

In [26]:
# Let's tokenizer a simple input
tokenizer.model = BPE('./train/vocab.json', './train/merges.txt')
encoding = tokenizer.encode("[CLS] start tempo120 melody:v100:A4 wait:12")

print("Encoded string: {}".format(encoding.tokens))

decoded = tokenizer.decode(encoding.ids)
print("Decoded string: {}".format(decoded))

Encoded string: ['Ġ[', 'cls', ']', 'Ġ', 's', 't', 'a', 'r', 't', 'Ġtempo', '120', 'Ġmelody', ':', 'v', '100', ':', 'a', '4', 'Ġwait', ':', '12']
Decoded string:  [cls] start tempo120 melody:v100:a4 wait:12


In [ ]:
def labeler(example, index):
  return example, tf.cast(index, tf.int64)  

labeled_data_sets = []

lines_dataset = tf.data.TextLineDataset(file_name)
labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, 0))
labeled_data_sets.append(labeled_dataset)

# for i, file_name in enumerate(FILE_NAMES):
#   lines_dataset = tf.data.TextLineDataset(os.path.join(parent_dir, file_name))
#   labeled_dataset = lines_dataset.map(lambda ex: labeler(ex, i))
#   labeled_data_sets.append(labeled_dataset)

In [ ]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000

all_labeled_data = labeled_data_sets[0]

# all_labeled_data = labeled_data_sets[0]
# for labeled_dataset in labeled_data_sets[1:]:
#   all_labeled_data = all_labeled_data.concatenate(labeled_dataset)
  
shuffled_labeled_data = all_labeled_data.shuffle(BUFFER_SIZE, reshuffle_each_iteration=False)

In [ ]:
# example
for ex in all_labeled_data.take(5):
  print(ex)

In [ ]:
tokenizer = tfds.features.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in all_labeled_data:
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)

vocab_size

print(vocabulary_set)

{'55', '135', '12', 'v78', 'accomp10', '111', '139', '38', 'F2', '1504', 'v48', '120', '21', 'v65', '79', 'v83', '110', 'v99', '96', '209', '125', 'v59', 'tempo91', '87', 'G4', '39', '767', 'v75', 'v89', '948', '411', '26', 'tempo126', '76', 'v63', 'v91', '11', 'v66', '100', '46', 'tempo86', '3875', '16', '205', '105', '13', 'tempo170', '455', 'v77', 'v7', 'tempo60', '41', 'v25', 'v71', '20', '298', 'melody', 'v97', '440', 'C3', 'tempo147', 'F4', '113', 'C5', '314', '102', '186', 'G3', 'v8', '45', '70', '960', '121', 'B4', 'D7', 'v53', '3632', '122', 'v32', 'v30', '104', '37', '71', 'E6', '1', 'tempo115', 'tempo175', 'C4', 'accomp6', '89', 'B', '815', '40', '95', '50', '577', '67', 'v88', 'v54', '181', '42', '151', 'v33', '74', '3647', 'v100', '996', 'v47', 'tempo87', 'tempo148', '303', 'v69', '866', '86', '191', '144', '22', '19', 'v18', '24', '383', 'tempo150', 'v60', 'tempo107', '69', 'v74', 'tempo104', 'tempo109', '10898', 'accomp1', 'C7', 'v119', '896', '345', '118', '864', '30', 

In [ ]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [ ]:
# example
example_text = next(iter(all_labeled_data))[0].numpy()
print(example_text)

b'[CLS] tempo128 [127] melody:v91:D4 melody:v91:B-3 melody:v91:F4 accomp0:v92:F2 wait:96 melody:v0:D4 melody:v0:B-3 melody:v0:F4 accomp0:v0:F2 accomp0:v92:G1 wait:96 melody:v91:F4 melody:v91:D4 melody:v91:B-3 accomp0:v0:G1 wait:48 melody:v0:B-3 melody:v0:F4 melody:v0:D4 melody:v91:D4 melody:v91:B-3 melody:v91:G4 wait:48 melody:v0:G4 melody:v0:B-3 melody:v0:D4 wait:24 melody:v91:D4 melody:v91:B-3 melody:v91:G4 wait:48 melody:v0:D4 melody:v0:B-3 melody:v0:G4 wait:24 melody:v91:B-3 melody:v91:D4 melody:v91:F4 accomp0:v92:F2 wait:96 [126] melody:v0:B-3 melody:v0:D4 melody:v0:F4 accomp0:v0:F2 accomp0:v92:G1 wait:96 melody:v91:F4 melody:v91:B-3 melody:v91:D4 accomp0:v0:G1 wait:48 melody:v0:D4 melody:v0:F4 melody:v0:B-3 melody:v91:D4 melody:v91:B-3 melody:v91:G4 wait:48 melody:v0:G4 melody:v0:B-3 melody:v0:D4 wait:24 melody:v91:D4 melody:v91:B-3 melody:v91:G4 wait:48 melody:v0:D4 melody:v0:B-3 melody:v0:G4 wait:24 melody:v91:B-3 melody:v91:D4 melody:v91:F4 accomp0:v92:F2 wait:96 melody:v0:B-3

In [ ]:
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

In [ ]:
def encode_map_fn(text, label):
  # py_func doesn't set the shape of the returned tensors.
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))

  # `tf.data.Datasets` work best if all components have a shape set
  #  so set the shapes manually: 
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


all_encoded_data = all_labeled_data.map(encode_map_fn)

In [ ]:
# example
example_text = next(iter(all_encoded_data))[0].numpy()
print(example_text)

[192 236 288 ... 273 188 143]


In [ ]:
train_data = all_encoded_data.skip(TAKE_SIZE).shuffle(BUFFER_SIZE)
train_data = train_data.padded_batch(BATCH_SIZE).repeat(30)

test_data = all_encoded_data.take(TAKE_SIZE)
test_data = test_data.padded_batch(BATCH_SIZE)

In [ ]:
test_data

<PaddedBatchDataset shapes: ((None, None), (None,)), types: (tf.int64, tf.int64)>

In [25]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./data/data_express_original.txt",
    block_size=128,
)

AttributeError: ignored

###### Training

In [ ]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Fix to TypeError: 
# You are attempting to use Python control flow in a layer that was not declared to be dynamic. Pass `dynamic=True` to the class constructor
model.run_eagerly = True

In [ ]:
# Train and evaluate using tf.keras.Model.fit()
history = model.fit(train_data, epochs=2) #steps_per_epoch=115,)
                    #validation_data=test_data, validation_steps=7)

Epoch 1/2


TypeError: ignored

In [ ]:
eval_loss, eval_acc = model.evaluate(test_data)
print('\nEval loss: {:.3f}, Eval accuracy: {:.3f}'.format(eval_loss, eval_acc))

In [ ]:
model.save_pretrained('./save/')

In [ ]:
model = tf.keras.Sequential()

# The first layer converts integer representations to dense vector embeddings
model.add(tf.keras.layers.Embedding(vocab_size, 64))

# The next layer is a Long Short-Term Memory layer, 
# which lets the model understand words in their context with other words. 
# A bidirectional wrapper on the LSTM helps it to learn about the datapoints 
# in relationship to the datapoints that came before it and after it.
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))

# a series of one or more densely connected layers, with the last one being the output layer. 
# The output layer produces a probability for all the labels. 
# The one with the highest probability is the models prediction of an example's label.
for units in [64, 64]:
  model.add(tf.keras.layers.Dense(units, activation='relu'))

# Output layer. The first argument is the number of labels.
model.add(tf.keras.layers.Dense(3))


model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_data, epochs=3, validation_data=test_data)

Epoch 1/3


TypeError: ignored

In [ ]:


special_tokens_dict = {'cls_token': '[CLS]',
                      'sep_token': '[SEP]',
                      }


num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print('Added new tokens: {}'.format(num_added_toks))
model.resize_token_embeddings(len(tokenizer))

train

In [ ]:
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states, mems = outputs[:2]